<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/%E8%A7%86%E9%A2%91%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install catboost

In [0]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import time
import lightgbm as lgb
import catboost as cat
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.model_selection import train_test_split,StratifiedKFold

#### 加载数据及简单分析

In [0]:
app_data = pd.read_csv('/content/drive/My Drive/比赛/Video/app.csv')
train_data = pd.read_csv('/content/drive/My Drive/比赛/Video/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/比赛/Video/test.csv')
user_data = pd.read_csv('/content/drive/My Drive/比赛/Video/user.csv')

In [0]:
 def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
    col_type = df[col].dtypes
    if col_type in numerics:
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [0]:
app_data = reduce_mem_usage(app_data)
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)
user_data = reduce_mem_usage(user_data)

Mem. usage decreased to  2.46 Mb (0.0% reduction)
Mem. usage decreased to 933.07 Mb (28.3% reduction)
Mem. usage decreased to 296.17 Mb (18.3% reduction)
Mem. usage decreased to  5.15 Mb (41.7% reduction)


In [0]:
app_data.to_pickle('/content/drive/My Drive/比赛/Video/app.pkl')
train_data.to_pickle('/content/drive/My Drive/比赛/Video/train.pkl')
test_data.to_pickle('/content/drive/My Drive/比赛/Video/test.pkl')
user_data.to_pickle('/content/drive/My Drive/比赛/Video/user.pkl')

##### 日期信息处理

In [0]:
def twosplit(x):
  if x >= 6 and x < 18:
    return 1
  else:
    return 0
def foursplit(x):
  if x >= 0 and x < 6:
    return 0
  elif x >= 6 and x < 12:
    return 1
  elif x >= 12 and x < 18:
    return 2
  elif x >= 18:
    return 3
def eightsplit(x):
  if x >= 0 and x < 3:
    return 0
  elif x >= 3 and x < 6:
    return 1
  elif x >= 6 and x < 9:
    return 2
  elif x >= 9 and x < 12:
    return 3
  elif x >= 12 and x < 15:
    return 4
  elif x >= 15 and x < 18:
    return 5
  elif x >= 18 and x < 21:
    return 6
  elif x >= 21:
    return 7
def dealVendor(x):
  x = x.lower()
  vendor = {'4g ':'4g','4g  ':'4g','oysin x5':'oysin','x-apple':'apple','iphone':'apple'}
  limit = ['vivo','oppo','huawei','honor','xiaomi','meizu','samsung','gionee','apple']
  if x in vendor:
    x = vendor[x]
    if x not in limit:
      return 'other'
    else:
      return x
  else:
    if x not in limit:
      return 'other'
    else:
      return x
def cleanAPP(x):
  x = x.replace('[','').replace(']','')[:-1]
  return set(x.split(' '))
def getMean(x):
  temp = 0
  for i in x:
    temp += i
  return temp/len(x)
def getTotal(x):
  temp = 0
  for i in x:
    temp += i
  return temp
def getTagRat(a,b):
  temp = []
  for i in range(len(a)):
    if len(a[i]) == 0 or len(b[i]) == 0:
      temp.append(0)
    else:
      temp.append((len(a[i]+b[i]) - len(set(a[i]+b[i])))/len(a[i]))
  return temp

#### APP信息

In [0]:
app_data = app_data.groupby('deviceid')['applist'].sum().reset_index()
app_data['applist'] = app_data['applist'].apply(cleanAPP)
app_data['len_applist'] = app_data['applist'].apply(lambda x: 1 if str(x) == 'nan' else len(x))
del app_data['applist']

In [0]:
len(set(app_data['deviceid'])),app_data.shape

(114584, (114584, 2))

#### User信息

In [0]:
user_data['gender'] = user_data['gender'].fillna(-1)
user_data['gender'] = user_data['gender'] + 1
user_data['personalscore'] = user_data['personalscore'].fillna(0)
user_data['personidentification'] = user_data['personidentification'].fillna(-1)
user_data['personidentification'] = user_data['personidentification'] + 1
user_data['level'] = user_data['level'].fillna(0)
user_data['followscore'] = user_data['followscore'].fillna(0)

user_data['has_tag'] = user_data['tag'].apply(lambda x: 0 if str(x) == 'nan' else 1)
user_data['tag_list'] = user_data['tag'].apply(lambda x: [] if str(x) == 'nan' else list(map(lambda y: y.split('_')[0],x.split('|'))))
user_data['tag_score'] = user_data['tag'].apply(lambda x: [0] if str(x) == 'nan' else list(map(lambda y: y.split('_cs:')[-1],x.split('|'))))
user_data['tag_score'] = user_data['tag_score'].apply(lambda x: list(map(lambda y: 0 if not str(y).replace('.','').isdigit() else float(y),x)))
user_data['len_tag'] = user_data['tag_list'].apply(lambda x: len(x))
user_data['tag_total'] = user_data['tag_score'].apply(getTotal)
user_data['tag_score_max'] = user_data['tag_score'].apply(lambda x: max(x))
user_data['tag_score_min'] = user_data['tag_score'].apply(lambda x: min(x))
user_data['tag_score_mean'] = user_data['tag_score'].apply(getMean)

user_data['has_outertag'] = user_data['outertag'].apply(lambda x: 0 if str(x) == 'nan' else 1)
user_data['outertag_list'] = user_data['outertag'].apply(lambda x: [] if str(x) == 'nan' else list(map(lambda y: y.split('_')[0],x.split('|'))))
user_data['outertag_score'] = user_data['outertag'].apply(lambda x: [0] if str(x) == 'nan' else list(map(lambda y: y.split('_cs:')[-1],x.split('|'))))
user_data['outertag_score'] = user_data['outertag_score'].apply(lambda x: list(map(lambda y: 0 if not str(y).replace('.','').isdigit() else float(y),x)))
user_data['len_outertag'] = user_data['outertag_list'].apply(lambda x: len(x))
user_data['outertag_total'] = user_data['outertag_score'].apply(getTotal)
user_data['outertag_score_max'] = user_data['outertag_score'].apply(lambda x: max(x))
user_data['outertag_score_min'] = user_data['outertag_score'].apply(lambda x: min(x))
user_data['outertag_score_mean'] = user_data['outertag_score'].apply(getMean)

user_data['outertag_tag'] = getTagRat(user_data['outertag_list'].values, user_data['tag_list'].values)
user_data['tag_outertag'] = getTagRat(user_data['tag_list'].values, user_data['outertag_list'].values)
user_data['has_guid'] = user_data['guid'].apply(lambda x: 0 if str(x) == 'nan' else 1)

user_data = user_data.drop(columns=['outertag','tag','tag_list','outertag_list','tag_score','outertag_score'])

In [0]:
user_data = user_data.groupby('deviceid').max().reset_index()

#### 基本信息

In [0]:
def transTime(data):
  data['date'] = data['ts'].apply(lambda x: str(datetime.utcfromtimestamp(x//1000)))
  data['days'] = data['date'].apply(lambda x: int(x.split(' ')[0].split('-')[-1]))
  data['dayofweek'] = data['days'].apply(lambda x: x%7)
  data['hour'] = data['date'].apply(lambda x: int(x.split(' ')[1].split(':')[0]))
  data['minute'] = data['date'].apply(lambda x: int(x.split(' ')[1].split(':')[-1]))
  data['twosplit'] = data['hour'].apply(twosplit)
  data['foursplit'] = data['hour'].apply(foursplit)
  data['eightsplit'] = data['hour'].apply(eightsplit)
  data['guid'] = data['guid'].fillna('loss')
  data['flag'] = data['days']
  
  return data
def featureEncoder(data):
  data['device_vendor'] = data['device_vendor'].apply(dealVendor)
  data['os'] =data['device_vendor'].apply(lambda x: 0 if x != 'apple' else 1)
  data['osversion'] = data['osversion'].apply(lambda x: int(x.split('.')[0]))
  data['app_big_version'] = data['app_version'].apply(lambda x: int(x.split('.')[0]))
  
  # 是否有位置信息
  data['has_location'] = data['lng'].apply(lambda x: 0 if x == 5e-324 else 1)
  # 异常点处理
  data['lng'] = data['lng'].apply(lambda x: 0.0 if x == 5e-324 else x)
  data['lat'] = data['lat'].apply(lambda x: 0.0 if x == 5e-324 else x)
  # 如果按城市划分的话，经纬度精确到个位已经差不多了
  data['lng'] = data['lng'].apply(lambda x: int(x))
  data['lat'] = data['lat'].apply(lambda x: int(x))
  data['location'] = data['lng'].astype(str)+' '+data['lat'].astype(str)
  encodecols = ['app_version','app_big_version','device_vendor','netmodel','osversion','device_version','location']
  for col in encodecols:
    encoder = LabelEncoder().fit(data[col])
    data[col] = encoder.transform(data[col])
  
  return data

In [0]:
train_data = transTime(train_data)
test_data = transTime(test_data)
test_data['flag'] = 11

In [0]:
data = pd.concat([train_data,test_data],axis=0,sort=False)
data = featureEncoder(data)
del train_data,test_data

In [0]:
history_9 = data[data['days']==8]
history_10 = data[data['days']==9]
history_11 = data[data['days']==10]
history_12 = data[data['days']==11]
del data

In [0]:
#喜爱度
def get_history_visit_time(data1,date2,features):
  data1 = data1.sort_values(['ts','timestamp'])
  data1['timestamp_ts'] = data1['timestamp'] - data1['ts']
  data1_tmp = data1[data1['target']==1].copy()
  del data1
  for col in features:
    print(col)
    for ts in ['timestamp_ts']:
      f_tmp = data1_tmp.groupby([col],as_index=False)[ts].agg({
          '{}_{}_max'.format(col,ts):'max',
          '{}_{}_mean'.format(col,ts):'mean',
          '{}_{}_min'.format(col,ts):'min',
          '{}_{}_median'.format(col,ts):'median',
          '{}_{}_std'.format(col,ts):'std',
      })
    date2 = pd.merge(date2,f_tmp,on=[col],how='left',copy=False)
  return date2

In [0]:
features = ['deviceid','guid','newsid','pos']
history_10 = reduce_mem_usage(get_history_visit_time(history_9,history_10,features))
history_11 = reduce_mem_usage(get_history_visit_time(history_10,history_11,features))
history_12 = reduce_mem_usage(get_history_visit_time(history_11,history_12,features))

deviceid
guid
newsid
pos
Mem. usage decreased to 584.92 Mb (45.8% reduction)
deviceid
guid
newsid
pos
Mem. usage decreased to 613.44 Mb (45.8% reduction)
deviceid
guid
newsid
pos
Mem. usage decreased to 491.69 Mb (42.5% reduction)


In [0]:
data = pd.concat([history_10,history_11],axis=0,sort=False,ignore_index=True)
data = pd.concat([data,history_12],axis=0,sort=False,ignore_index=True)
del history_9,history_10,history_11,history_12

In [0]:
for col in [['deviceid'],['guid'],['newsid'],['pos']]:
  print(col)
  data['{}_days_count'.format('_'.join(col))] = data.groupby(['days'] + col)['id'].transform('count')

['deviceid']
['guid']
['newsid']
['pos']


In [0]:
data = pd.merge(data,app_data,on='deviceid',how='left',sort=False)
data = pd.merge(data,user_data,on='deviceid',how='left',sort=False)
data = reduce_mem_usage(data)
del app_data,user_data

Mem. usage decreased to 2191.72 Mb (35.4% reduction)


#### 缓存

In [0]:
data.to_pickle('/content/drive/My Drive/比赛/Video/data.pkl')

In [0]:
data = pd.read_pickle('/content/drive/My Drive/比赛/Video/data.pkl')

In [0]:
data.shape

(10689253, 73)

#### 数据集划分

In [0]:
X_train = data[data['flag'].isin([9])]
X_valid = data[data['flag'].isin([10])]
X_test = data[data['flag'].isin([11])]
X_train.shape,X_valid.shape,X_test.shape

((3762779, 73), (3272882, 73), (3653592, 73))

In [0]:
# history_10.to_pickle('/content/drive/My Drive/比赛/Video/history_10.pkl')
# history_11.to_pickle('/content/drive/My Drive/比赛/Video/history_11.pkl')
# history_12.to_pickle('/content/drive/My Drive/比赛/Video/history_12.pkl')

In [0]:
#((3762779, 64), (3272882, 64), (3653592, 64))
X_train.columns

Index(['id', 'target', 'timestamp', 'deviceid', 'newsid', 'guid', 'pos',
       'app_version', 'device_vendor', 'netmodel', 'osversion', 'lng', 'lat',
       'device_version', 'ts', 'date', 'days', 'dayofweek', 'hour', 'minute',
       'twosplit', 'foursplit', 'eightsplit', 'flag', 'os', 'app_big_version',
       'has_location', 'location', 'deviceid_timestamp_ts_max',
       'deviceid_timestamp_ts_mean', 'deviceid_timestamp_ts_min',
       'deviceid_timestamp_ts_median', 'deviceid_timestamp_ts_std',
       'guid_timestamp_ts_max', 'guid_timestamp_ts_mean',
       'guid_timestamp_ts_min', 'guid_timestamp_ts_median',
       'guid_timestamp_ts_std', 'newsid_timestamp_ts_max',
       'newsid_timestamp_ts_mean', 'newsid_timestamp_ts_min',
       'newsid_timestamp_ts_median', 'newsid_timestamp_ts_std',
       'pos_timestamp_ts_max', 'pos_timestamp_ts_mean', 'pos_timestamp_ts_min',
       'pos_timestamp_ts_median', 'pos_timestamp_ts_std',
       'deviceid_days_count', 'guid_days_count', 'new

#### 训练模型

In [0]:
lgb_params = {
  'objective': 'binary',
  'learning_rate': 0.1,
  'num_leaves': 31,
  'bagging_fraction': .9,
  'feature_fraction': .9,
  'lambda_l1': 0.1,
  'metric': 'auc',
  'seed': 2019,
  'max_depth': -1,
}
def eval_func(y_pred, train_data):
  y_true = train_data.get_label()
  score = f1_score(y_true, np.round(y_pred))
  return 'f1', score, True

In [0]:
feature = [ col for col in list(X_train.columns) if col not in ['id','target','timestamp','newsid','deviceid','guid','ts_next','ts','date','flag']]
target = 'target'
cat_features = ['pos','app_version','device_vendor', 'netmodel', 'osversion',
         'device_version', 'dayofweek','twosplit', 'foursplit', 'eightsplit',
          'os', 'app_big_version', 'has_location', 'location',
         'level', 'gender', 'has_tag', 'has_outertag', 'has_guid']

In [0]:
lgb_train = lgb.Dataset(X_train[feature].values, X_train[target].values)
lgb_valid = lgb.Dataset(X_valid[feature].values, X_valid[target].values, reference=lgb_train)

##### 验证模型

In [0]:
lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=10000, valid_sets=[lgb_train,lgb_valid], early_stopping_rounds=50, verbose_eval=5,)

Training until validation scores don't improve for 50 rounds.
[5]	training's auc: 0.894306	valid_1's auc: 0.884244
[10]	training's auc: 0.898568	valid_1's auc: 0.886389
[15]	training's auc: 0.902427	valid_1's auc: 0.890897
[20]	training's auc: 0.905707	valid_1's auc: 0.894406
[25]	training's auc: 0.907676	valid_1's auc: 0.896311
[30]	training's auc: 0.909268	valid_1's auc: 0.898116
[35]	training's auc: 0.910747	valid_1's auc: 0.89958
[40]	training's auc: 0.911968	valid_1's auc: 0.90058
[45]	training's auc: 0.912923	valid_1's auc: 0.901417
[50]	training's auc: 0.913837	valid_1's auc: 0.902075
[55]	training's auc: 0.91474	valid_1's auc: 0.90261
[60]	training's auc: 0.915494	valid_1's auc: 0.903048
[65]	training's auc: 0.916165	valid_1's auc: 0.90347
[70]	training's auc: 0.916745	valid_1's auc: 0.903779
[75]	training's auc: 0.917342	valid_1's auc: 0.904106
[80]	training's auc: 0.917828	valid_1's auc: 0.904372
[85]	training's auc: 0.918273	valid_1's auc: 0.904559
[90]	training's auc: 0.918

In [0]:
# 227
p_test = lgb_model.predict(X_valid[feature].values,num_iteration=lgb_model.best_iteration)
xx_score = X_valid[[target]].copy()
xx_score['predict'] = p_test
xx_score = xx_score.sort_values('predict',ascending=False)
xx_score = xx_score.reset_index()
xx_score.loc[xx_score.index<=int(xx_score.shape[0]*0.110),'score'] = 1
xx_score['score'] = xx_score['score'].fillna(0)
print(f1_score(xx_score['target'],xx_score['score']))

0.5431915347368633


In [0]:
X_train = data[data['flag'].isin([9,10])]
# del lgb_train,lgb_valid
lgb_train = lgb.Dataset(X_train[feature].values, X_train[target].values)

In [0]:
lgb_model_2 = lgb.train(lgb_params, lgb_train, num_boost_round=510, valid_sets=[lgb_train],verbose_eval=10)

[10]	training's auc: 0.895171
[20]	training's auc: 0.901581
[30]	training's auc: 0.905327
[40]	training's auc: 0.90806
[50]	training's auc: 0.910022
[60]	training's auc: 0.91148
[70]	training's auc: 0.912651
[80]	training's auc: 0.913603
[90]	training's auc: 0.914461
[100]	training's auc: 0.91515
[110]	training's auc: 0.915785
[120]	training's auc: 0.916329
[130]	training's auc: 0.916865
[140]	training's auc: 0.917345
[150]	training's auc: 0.917776
[160]	training's auc: 0.918163
[170]	training's auc: 0.918515
[180]	training's auc: 0.918897
[190]	training's auc: 0.919248
[200]	training's auc: 0.919629
[210]	training's auc: 0.919946
[220]	training's auc: 0.920284
[230]	training's auc: 0.920585
[240]	training's auc: 0.920833
[250]	training's auc: 0.921113
[260]	training's auc: 0.921413
[270]	training's auc: 0.921651
[280]	training's auc: 0.921908
[290]	training's auc: 0.92212
[300]	training's auc: 0.922401
[310]	training's auc: 0.922638
[320]	training's auc: 0.922893
[330]	training's auc:

In [0]:
del X_train,lgb_train

In [0]:
p_predict = lgb_model_2.predict(X_test[feature].values)

submit_score = X_test[['id']].copy()
submit_score['predict'] = p_predict
submit_score = submit_score.sort_values('predict',ascending=False)
submit_score = submit_score.reset_index()
submit_score.loc[submit_score.index<=int(submit_score.shape[0]*0.110),'target'] = 1
submit_score['target'] = submit_score['target'].fillna(0)

submit_score = submit_score.sort_values('id')
submit_score['target'] = submit_score['target'].astype(int)

sample = pd.read_csv('/content/drive/My Drive/比赛/Video/sample.csv')
sample.columns = ['id','non_target']
submit_score = pd.merge(sample,submit_score,on=['id'],how='left')

submit_score[['id','target']].to_csv('baseline.csv',index=False)

In [0]:
submit_score.shape

(3653592, 5)

In [0]:
X_train = data[data['flag'].isin([9])]
X_valid = data[data['flag'].isin([10])]
X_test = data[data['flag'].isin([11])]

In [0]:
eval_set = cat.Pool(data=X_valid.iloc[feature],label=X_valid.iloc[target],cat_features=cat_features)

In [0]:
model = cat.CatBoostClassifier(iterations=5000, learning_rate=0.1, depth=12, loss_function='Logloss', eval_metric='AUC')

In [0]:
model.fit(X_train[feature],X_train[target],use_best_model=True,eval_set=eval_set,cat_features=cat_features,early_stopping_rounds=50)

0:	test: 0.9018846	best: 0.9018846 (0)	total: 24.4s	remaining: 1d 9h 54m 29s
1:	test: 0.9059395	best: 0.9059395 (1)	total: 47.1s	remaining: 1d 8h 40m 24s
2:	test: 0.9086644	best: 0.9086644 (2)	total: 1m 8s	remaining: 1d 7h 36m 50s
3:	test: 0.9094462	best: 0.9094462 (3)	total: 1m 32s	remaining: 1d 8h 11m 35s
4:	test: 0.9099553	best: 0.9099553 (4)	total: 1m 56s	remaining: 1d 8h 22m 40s
5:	test: 0.9111612	best: 0.9111612 (5)	total: 2m 18s	remaining: 1d 8h 3m 40s
6:	test: 0.9121002	best: 0.9121002 (6)	total: 2m 40s	remaining: 1d 7h 43m 59s
7:	test: 0.9120915	best: 0.9121002 (6)	total: 3m 3s	remaining: 1d 7h 45m 10s
8:	test: 0.9126819	best: 0.9126819 (8)	total: 3m 24s	remaining: 1d 7h 32m 1s
9:	test: 0.9129804	best: 0.9129804 (9)	total: 3m 48s	remaining: 1d 7h 36m 39s
10:	test: 0.9137110	best: 0.9137110 (10)	total: 4m 10s	remaining: 1d 7h 34m 48s
11:	test: 0.9140861	best: 0.9140861 (11)	total: 4m 32s	remaining: 1d 7h 26m 57s
12:	test: 0.9146020	best: 0.9146020 (12)	total: 4m 55s	remaining: 

##########################################################################################

app:(161417, 2)
> deviceid 设备ID

> applist 设备中APP列表

train:(11376681, 15)

> id 记录ID

> target 记录Label

> timestamp 时间戳，代表改用户点击该视频的时间戳，如果未点击则为NULL

> deviceid 设备id

> newsid 视频的id

> guid 用户的注册id

> pos 视频推荐位置

> app_version APP版本

> device_vendor 设备供应商

> netmodel 网络模式

> osversion 操作系统

> lng 经度

> lat 维度

> device_version 设备版本

> ts 视频暴光给用户的时间戳

test:(3653592, 13)

> id

> deviceid

> newsid

> guid

> pos

> app_version

> device_vendor

> netmodel

> osversion

> lng

> lat

> device_version

> ts

user:(128573, 9)

> deviceid 设备id

> guid 全局位移标识符

> outertag 对输出话题的兴趣值

> tag 对话题的兴趣值

> level 用户等级

> personidentification 是否有识别信息的用户

> followscore 关注分数

> personalscore 对用户的评分，还有负分

> gender 性别